In [22]:
import matplotlib.pyplot as py
import scraper
from pprint import pprint
import numpy as np

In [37]:
def priceChanges(prices):
    absChange = [0]*(len(prices)-1)
    percChange = [0]*(len(prices)-1)
    for i in range(0,len(prices)-1):
        absChange[i] = prices[i+1]-prices[i]
        percChange[i] = absChange[i]/prices[i]
    return absChange,percChange

def marginChanges(prices,buylimit):
    absChange = [0]*(len(prices)-1)
    percChange = [0]*(len(prices)-1)
    for i in range(0,len(prices)-1):
        absChange[i] = prices[i+1]*buylimit-prices[i]*buylimit
        percChange[i] = absChange[i]/(prices[i]*buylimit)
    return absChange,percChange

def getPerfectProfit(data):
    buyers = [10**5,10**6,10**7,10**8,10**9,10**10]
    profit = [0] * len(buyers)
    count = [0] * len(buyers)
    for i in range(1,len(data)):
        if data[i]>data[i-1]:
            for b in range(len(buyers)):
                amount = int(buyers[b]/data[i-1])
                profit[b] += (data[i]-data[i-1])*amount
                count[b] += amount
    for b in range(len(buyers)):
        if count[b] == 0:
            profit[b] == 0
        else:
            profit[b]/=count[b]
    return profit

In [25]:
itemInfo = scraper.loadItemInfo()
itemPrices = scraper.loadItemPrices()

for item in itemPrices:
    prices = [a[1] for a in itemPrices[item]]
    mean = np.mean(prices)
    std = np.std(prices)
    itemPrices[item] = [itemPrices[item][i] for i in range(len(prices)) if prices[i]>mean+2*std]

In [52]:
profits = []
for item in itemPrices:
    profits.append((getPerfectProfit([a[1] for a in itemPrices[item]]),item))

def getKey(item):
    return item[0][1]
    
profits.sort(key=getKey,reverse=True)

In [53]:
pprint(profits[:10])

[([0,
   164214.8108108108,
   171809.68690702086,
   171307.0640339546,
   171299.93000312897,
   171295.10883592733],
  'Uncharged_trident'),
 ([0,
   113034.9074074074,
   110978.94416243654,
   110979.37189460477,
   110973.63892022899,
   110972.71401854164],
  'Dragon_sq_shield_ornament_kit'),
 ([0,
   91440.0,
   50405.45038167939,
   50381.60975609756,
   50373.816554969344,
   50374.10837638508],
  'Black_gold-trimmed_set_(lg)'),
 ([0,
   54920.011235955055,
   54799.961127308066,
   54750.72887526346,
   54733.57826565265,
   54732.76480147374],
  'Team-11_cape'),
 ([0,
   48825.0,
   46073.3698630137,
   46038.08355091384,
   46047.380475757185,
   46047.52635545104],
  'Iron_platebody_(g)'),
 ([0,
   43046.0,
   35638.05263157895,
   35516.994897959186,
   35491.064401622716,
   35490.943779783025],
  'Elder_chaos_top'),
 ([0,
   40802.0,
   47408.90909090909,
   47496.92505353319,
   47503.03765957447,
   47502.5322117532],
  'Evil_chicken_head'),
 ([0,
   40373.0,
   4428